Lesson 2: Sequential Chats and Customer Onboarding

In [3]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
llm_config = {"model": "gpt-3.5-turbo"}

Creating the needed agents

In [4]:
from autogen import ConversableAgent

In [5]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [6]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [7]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [13]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

Creating tasks: Now, you can craft a series of tasks to facilitate the onboarding process.

In [14]:
#Sequential chats between two agents
#will go on till max turns are done or termination is reached

chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

Start the onboarding process


In [15]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding Personal Information Agent):



--------------------------------------------------------------------------------
Onboarding Personal Information Agent (to customer_proxy_agent):

I'm sorry for the inconvenience, but in order to assist you further, could you please provide me with your name and location?

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding Personal Information Agent):



--------------------------------------------------------------------------

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/autogen/agentchat/chat.py:47: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Print out the summary

In [16]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{
  "name": "John",
  "location": "New York"
}


John is located in New York and is interested in reading about various topics.


John from New York is interested in reading about the Empire State Building, including fun facts such as its quick construction time and impressive visibility from the top.




In [17]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.000317, 'gpt-3.5-turbo-0125': {'cost': 0.000317, 'prompt_tokens': 376, 'completion_tokens': 86, 'total_tokens': 462}}, 'usage_excluding_cached_inference': {'total_cost': 0.0001585, 'gpt-3.5-turbo-0125': {'cost': 0.0001585, 'prompt_tokens': 188, 'completion_tokens': 43, 'total_tokens': 231}}}


{'usage_including_cached_inference': {'total_cost': 0.000111, 'gpt-3.5-turbo-0125': {'cost': 0.000111, 'prompt_tokens': 132, 'completion_tokens': 30, 'total_tokens': 162}}, 'usage_excluding_cached_inference': {'total_cost': 5.55e-05, 'gpt-3.5-turbo-0125': {'cost': 5.55e-05, 'prompt_tokens': 66, 'completion_tokens': 15, 'total_tokens': 81}}}


{'usage_including_cached_inference': {'total_cost': 0.0004904999999999999, 'gpt-3.5-turbo-0125': {'cost': 0.0004904999999999999, 'prompt_tokens': 393, 'completion_tokens': 196, 'total_tokens': 589}}, 'usage_excluding_cached_inference': {'total_cost': 0.000307, 'gpt-3.5-turbo-0125': {'cost': 0.000307, 'pro